# OpenAI埋め込みのベクトルデータベースとしてのHologresの使用

このノートブックでは、OpenAI埋め込みのベクトルデータベースとしてHologresを使用する方法を段階的にガイドします。

このノートブックでは、以下のエンドツーエンドプロセスを紹介します：
1. OpenAI APIによって作成された事前計算済み埋め込みの使用
2. クラウドインスタンスのHologresへの埋め込みの保存
3. OpenAI APIを使用した生テキストクエリの埋め込みへの変換
4. Hologresを使用した作成されたコレクションでの最近傍探索の実行
5. プロンプトエンジニアリングにおけるコンテキストとして検索結果を大規模言語モデルに提供

### Hologresとは

[Hologres](https://www.alibabacloud.com/help/en/hologres/latest/what-is-hologres)は、Alibaba Cloudによって開発された統合リアルタイムデータウェアハウジングサービスです。Hologresを使用して、大量のデータをリアルタイムで書き込み、更新、処理、分析することができます。Hologresは標準SQL構文をサポートし、PostgreSQLと互換性があり、ほとんどのPostgreSQL機能をサポートしています。Hologresは最大ペタバイト規模のデータに対するオンライン分析処理（OLAP）とアドホック分析をサポートし、高い同時実行性と低遅延のオンラインデータサービスを提供します。Hologresは複数のワークロードのきめ細かい分離とエンタープライズレベルのセキュリティ機能をサポートしています。HologresはMaxCompute、Realtime Compute for Apache Flink、DataWorksと深く統合されており、企業向けのフルスタックオンライン・オフラインデータウェアハウジングソリューションを提供します。

Hologresは[Proxima](https://www.alibabacloud.com/help/en/hologres/latest/vector-processing)を採用することでベクトルデータベース機能を提供します。

Proximaは、Alibaba DAMO Academyによって開発された高性能ソフトウェアライブラリです。ベクトルの最近傍を検索することができます。ProximaはFacebook AI Similarity Search（Faiss）などの類似のオープンソースソフトウェアよりも高い安定性とパフォーマンスを提供します。Proximaは業界をリードするパフォーマンスと効果を持つ基本モジュールを提供し、類似の画像、動画、または人の顔を検索することができます。Hologresは高性能ベクトル検索サービスを提供するためにProximaと深く統合されています。

### デプロイメントオプション

- [Hologresデータウェアハウス](https://www.alibabacloud.com/help/en/hologres/latest/getting-started)を高速デプロイするには[こちらをクリック](https://www.alibabacloud.com/product/hologres)してください。

## 前提条件

この演習を行うために、いくつかの準備が必要です：

1. Hologresクラウドサーバーインスタンス
2. ベクトルデータベースとやり取りするための'psycopg2-binary'ライブラリ。その他のPostgreSQLクライアントライブラリでも構いません。
3. [OpenAI APIキー](https://beta.openai.com/account/api-keys)

サーバーが正常に起動したかどうかを確認するために、簡単な curl コマンドを実行して検証することができます：

### 要件のインストール

このノートブックには明らかに`openai`と`psycopg2-binary`パッケージが必要ですが、使用する他の追加ライブラリもいくつかあります。以下のコマンドでそれらすべてをインストールできます：

In [ ]:
! pip install openai psycopg2-binary pandas wget

### OpenAI APIキーの準備

OpenAI APIキーは、ドキュメントとクエリのベクトル化に使用されます。

OpenAI APIキーをお持ちでない場合は、[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)から取得できます。

キーを取得したら、環境変数に`OPENAI_API_KEY`として追加してください。

In [2]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


## Hologresに接続する
まず環境変数に追加してください。または、以下の"psycopg2.connect"パラメータを直接変更することもできます。

実行中のHologresサーバーインスタンスへの接続は、公式のPythonライブラリを使用すると簡単に行えます：

In [3]:
import os
import psycopg2

# Note. alternatively you can set a temporary env variable like this:
# os.environ["PGHOST"] = "your_host"
# os.environ["PGPORT"] "5432"),
# os.environ["PGDATABASE"] "postgres"),
# os.environ["PGUSER"] "user"),
# os.environ["PGPASSWORD"] "password"),

connection = psycopg2.connect(
    host=os.environ.get("PGHOST", "localhost"),
    port=os.environ.get("PGPORT", "5432"),
    database=os.environ.get("PGDATABASE", "postgres"),
    user=os.environ.get("PGUSER", "user"),
    password=os.environ.get("PGPASSWORD", "password")
)
connection.set_session(autocommit=True)

# Create a new cursor object
cursor = connection.cursor()

利用可能な任意のメソッドを実行して接続をテストできます：

In [4]:

# Execute a simple query to test the connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Connection successful!")
else:
    print("Connection failed.")

Connection successful!


In [ ]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

ダウンロードしたファイルは、その後解凍する必要があります：

In [6]:
import zipfile
import os
import re
import tempfile

current_directory = os.getcwd()
zip_file_path = os.path.join(current_directory, "vector_database_wikipedia_articles_embedded.zip")
output_directory = os.path.join(current_directory, "../../data")

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_directory)


# check the csv file exist
file_name = "vector_database_wikipedia_articles_embedded.csv"
data_directory = os.path.join(current_directory, "../../data")
file_path = os.path.join(data_directory, file_name)


if os.path.exists(file_path):
    print(f"The file {file_name} exists in the data directory.")
else:
    print(f"The file {file_name} does not exist in the data directory.")


The file vector_database_wikipedia_articles_embedded.csv exists in the data directory.


## データの読み込み

このセクションでは、このセッションの前に準備されたデータを読み込みます。これにより、あなた自身のクレジットを使ってWikipedia記事の埋め込みを再計算する必要がありません。

In [16]:
!unzip -n vector_database_wikipedia_articles_embedded.zip
!ls -lh vector_database_wikipedia_articles_embedded.csv

Archive:  vector_database_wikipedia_articles_embedded.zip
-rw-r--r--@ 1 geng  staff   1.7G Jan 31 01:19 vector_database_wikipedia_articles_embedded.csv


データを確認してください。

In [21]:
import pandas, json
data = pandas.read_csv('../../data/vector_database_wikipedia_articles_embedded.csv')
data

,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4
...,...,...,...,...,...,...,...
24995,98295,https://simple.wikipedia.org/wiki/Geneva,Geneva,"Geneva (, , , , ) is the second biggest cit...","[-0.015773078426718712, 0.01737344264984131, 0...","[0.008000412955880165, 0.02008531428873539, 0....",24995
24996,98316,https://simple.wikipedia.org/wiki/Concubinage,Concubinage,Concubinage is the state of a woman in a relat...,"[-0.00519518880173564, 0.005898841191083193, 0...","[-0.01736736111342907, -0.002740012714639306, ...",24996
24997,98318,https://simple.wikipedia.org/wiki/Mistress%20%...,Mistress (lover),A mistress is a man's long term female sexual ...,"[-0.023164259269833565, -0.02052430994808674, ...","[-0.017878392711281776, -0.0004517830966506153...",24997
24998,98326,https://simple.wikipedia.org/wiki/Eastern%20Front,Eastern Front,Eastern Front can be one of the following:\n\n...,"[-0.00681863259524107, 0.002171179046854377, 8...","[-0.0019235472427681088, -0.004023272544145584...",24998


In [26]:
title_vector_length = len(json.loads(data['title_vector'].iloc[0]))
content_vector_length = len(json.loads(data['content_vector'].iloc[0]))

print(title_vector_length, content_vector_length)

1536 1536


### テーブルとproxima vectorインデックスの作成

Hologresは__テーブル__にデータを格納し、各オブジェクトは少なくとも1つのベクトルで記述されます。私たちのテーブルは**articles**と呼ばれ、各オブジェクトは**title**と**content**の両方のベクトルで記述されます。

まず、テーブルを作成し、**title**と**content**の両方にproximaインデックスを作成してから、事前に計算された埋め込みでテーブルを埋めていきます。

In [3]:
cursor.execute('CREATE EXTENSION IF NOT EXISTS proxima;')
create_proxima_table_sql = '''
BEGIN;
DROP TABLE IF EXISTS articles;
CREATE TABLE articles (
    id INT PRIMARY KEY NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector float4[] check(
        array_ndims(title_vector) = 1 and 
        array_length(title_vector, 1) = 1536
    ), -- define the vectors
    content_vector float4[] check(
        array_ndims(content_vector) = 1 and 
        array_length(content_vector, 1) = 1536
    ),
    vector_id INT
);

-- Create indexes for the vector fields.
call set_table_property(
    'articles',
    'proxima_vectors', 
    '{
        "title_vector":{"algorithm":"Graph","distance_method":"Euclidean","builder_params":{"min_flush_proxima_row_count" : 10}},
        "content_vector":{"algorithm":"Graph","distance_method":"Euclidean","builder_params":{"min_flush_proxima_row_count" : 10}}
    }'
);  

COMMIT;
'''

# Execute the SQL statements (will autocommit)
cursor.execute(create_proxima_table_sql)

### データのアップロード

それでは、[COPY文](https://www.alibabacloud.com/help/en/hologres/latest/use-the-copy-statement-to-import-or-export-data)を使用してHologresクラウドインスタンスにデータをアップロードしましょう。ネットワーク帯域幅によって、これには5〜10分程度かかる場合があります。

In [4]:
import io

# Path to the unzipped CSV file
csv_file_path = '../../data/vector_database_wikipedia_articles_embedded.csv'

# In SQL, arrays are surrounded by {}, rather than []
def process_file(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            # Replace '[' with '{' and ']' with '}'
            modified_line = line.replace('[', '{').replace(']', '}')
            yield modified_line

# Create a StringIO object to store the modified lines
modified_lines = io.StringIO(''.join(list(process_file(csv_file_path))))

# Create the COPY command for the copy_expert method
copy_command = '''
COPY public.articles (id, url, title, content, title_vector, content_vector, vector_id)
FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ',');
'''

# Execute the COPY command using the copy_expert method
cursor.copy_expert(copy_command, modified_lines)

proximaインデックスはバックグラウンドで構築されます。この期間中も検索は可能ですが、ベクトルインデックスがないとクエリは遅くなります。インデックスの構築完了を待つには、このコマンドを使用してください。

In [5]:
cursor.execute('vacuum articles;')

In [6]:
# Check the collection size to make sure all the points have been stored
count_sql = "select count(*) from articles;"
cursor.execute(count_sql)
result = cursor.fetchone()
print(f"Count:{result[0]}")


Count:25000


## データの検索

データがアップロードされたら、最も近いベクトルを求めてコレクションにクエリを開始します。タイトルベースの検索からコンテンツベースの検索に切り替えるために、追加のパラメータ `vector_name` を提供することができます。事前計算された埋め込みは `text-embedding-3-small` OpenAIモデルで作成されたため、検索時にも同じモデルを使用する必要があります。

In [11]:
import openai
def query_knn(query, table_name, vector_name="title_vector", top_k=20):

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(
        input=query,
        model="text-embedding-3-small",
    )["data"][0]["embedding"]

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "{" + ",".join(map(str, embedded_query)) + "}"

    # Create SQL query
    query_sql = f"""
    SELECT id, url, title, pm_approx_euclidean_distance({vector_name},'{embedded_query_pg}'::float4[]) AS distance
    FROM {table_name}
    ORDER BY distance
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

In [13]:
query_results = query_knn("modern art in Europe", "Articles")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Museum of Modern Art (Score: 0.501)
2. Western Europe (Score: 0.485)
3. Renaissance art (Score: 0.479)
4. Pop art (Score: 0.472)
5. Northern Europe (Score: 0.461)
6. Hellenistic art (Score: 0.458)
7. Modernist literature (Score: 0.447)
8. Art film (Score: 0.44)
9. Central Europe (Score: 0.439)
10. Art (Score: 0.437)
11. European (Score: 0.437)
12. Byzantine art (Score: 0.436)
13. Postmodernism (Score: 0.435)
14. Eastern Europe (Score: 0.433)
15. Cubism (Score: 0.433)
16. Europe (Score: 0.432)
17. Impressionism (Score: 0.432)
18. Bauhaus (Score: 0.431)
19. Surrealism (Score: 0.429)
20. Expressionism (Score: 0.429)


In [14]:
# This time we'll query using content vector
query_results = query_knn("Famous battles in Scottish history", "Articles", "content_vector")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Battle of Bannockburn (Score: 0.489)
2. Wars of Scottish Independence (Score: 0.474)
3. 1651 (Score: 0.457)
4. First War of Scottish Independence (Score: 0.452)
5. Robert I of Scotland (Score: 0.445)
6. 841 (Score: 0.441)
7. 1716 (Score: 0.441)
8. 1314 (Score: 0.429)
9. 1263 (Score: 0.428)
10. William Wallace (Score: 0.426)
11. Stirling (Score: 0.419)
12. 1306 (Score: 0.419)
13. 1746 (Score: 0.418)
14. 1040s (Score: 0.414)
15. 1106 (Score: 0.412)
16. 1304 (Score: 0.411)
17. David II of Scotland (Score: 0.408)
18. Braveheart (Score: 0.407)
19. 1124 (Score: 0.406)
20. July 27 (Score: 0.405)
